#### Goal :: Predict the sale (number of items) for the shop-item pair in the  test set. The test set is for one month - November 2015 

In [62]:
import numpy as np
import pandas as pd
import os
import csv
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

#### Read data

In [63]:
DATA_FOLDER = './'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))

test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv.gz'))

In [64]:
print("Total Items: {}".format(items.item_id.nunique()))
print("Total Categories: {}".format(item_categories.item_category_id.nunique()))
print("Total Shops: {}".format(shops.shop_id.nunique()))
print("\n--------------------------------------------------------\n")

print("Training data")
print("  # unique shops: {}".format(transactions.shop_id.nunique()))
print("  # unique items: {}".format(transactions.item_id.nunique()))

print("\n--------------------------------------------------------\n")
print("Test data")
print("  # unique shops: {}".format(test.shop_id.nunique()))
print("  # unique items: {}".format(test.item_id.nunique()))


%matplotlib inline

# Merging to get category_id in the transaction and test df
transactions = pd.merge(transactions, items[['item_id', 'item_category_id']], on='item_id', how='inner')
test_mod = pd.merge(test, items[['item_id', 'item_category_id']], on='item_id', how='inner')

unique_categories, count = np.unique(items.item_category_id, return_counts=True)
ax = plt.subplot()
ax.bar(unique_categories, count)    
plt.title( "Distribution of Categories across all items")
plt.show()

unique_categories_trans, count_trans = np.unique(transactions.item_category_id, return_counts=True)
ax = plt.subplot()
ax.bar(unique_categories_trans, count_trans)
plt.title("Distribution of Categories across all transactions")
plt.show()

unique_categories_test, count_test = np.unique(test_mod.item_category_id, return_counts=True)
ax = plt.subplot()
ax.bar(unique_categories_test, count_test)
plt.title("Distribution of Categories across test dataset")
plt.show()

print("\n\n")

items_test_nottrans = len(set(test.item_id.unique()) - set(test.item_id.unique()).intersection(set(transactions.item_id.unique())))
intersection_items_test_transac = len(set(test.item_id.unique()).intersection(set(transactions.item_id.unique())))
item_count_test = len(set(test.item_id.unique()))

print("# items in test set: {}".format(item_count_test))
print("# items in test set and transaction set: {}".format(intersection_items_test_transac))
print("# items in test but not transaction: {}".format(items_test_nottrans))

print("\n")

shops_test_nottrans = len(set(test.shop_id.unique()) - set(test.shop_id.unique()).intersection(set(transactions.shop_id.unique())))
intersection_shops_test_transac = len(set(test.shop_id.unique()).intersection(set(transactions.shop_id.unique())))
shop_count_test = len(set(test.shop_id.unique()))

print("# shops in test set: {}".format(shop_count_test))
print("# shops in test set and transaction set: {}".format(intersection_shops_test_transac))
print("# shops in test but not transaction: {}".format(shops_test_nottrans))

print("\n")

# Distribution of items across months

#Get the items which are common to test and transaction data.
#For each item:
#    How many months have at least one entry for that item?

# Items present in both test and transaction data


item_distr = transactions.groupby(['item_id'])['date_block_num'].unique().reset_index(name='list_months')
item_distr["num_months"] = item_distr.list_months.apply(lambda x: x.size)
item_distr = item_distr.drop(['list_months'], axis=1)
item_distr_testset = item_distr[item_distr.item_id.isin(list(s))]

###### Constructing previous_value_benchmark

For the each shop/item pair our predictions are just monthly sales from the previous month, i.e. October 2015.

The most important step at reproducing this score is correctly aggregating daily data and constructing monthly sales data frame. You need to get lagged values, fill NaNs with zeros and clip the values into [0,20] range. If you do it correctly, you'll get precisely 1.16777 on the public leaderboard.

Generating features like this is a necessary basis for more complex models. Also, if you decide to fit some model, don't forget to clip the target into [0,20] range, it makes a big difference.

In [ ]:
def get_item_cnt(shopid, itemid):
    if (shopid, itemid) in trans_oct_grouped:
        count  = trans_oct_grouped[int(shopid), int(itemid)]
        if(count > 20):
            return 20
        elif(count < 0):
            return 0
        else:
            return count
    else:
        return 0

trans_oct = transactions[transactions.date_block_num == 33]
trans_oct_grouped = trans_oct.groupby(["shop_id", "item_id"]).item_cnt_day.sum()
previous_value_benchmark_submission = test

previous_value_benchmark_submission["item_cnt_month"] = previous_value_benchmark_submission.apply(lambda x: get_item_cnt(x.shop_id, x.item_id), axis=1)
print(previous_value_benchmark_submission.item_cnt_month.describe())

# writing previous_value_benchmark_submission ---> csv file
previous_value_benchmark_submission[["ID", "item_cnt_month"]].to_csv("previous_value_benchmark_submission", index=False)

###### Preprocess transaction (train) and test dataset

There are some items in test set which do not exist in transaction dataset. 
However, these items will probabaly be similar to some other items in the transaction dataset.
We can use this simillarity to predict for these items

In [65]:
# Drop the rows with price lt zero. ther is only one.
transactions = transactions[transactions.item_price > 0]

In [67]:
# Group the data by month and drop the date 
transactions.head(3)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


In [26]:
# code to get days, month, year
# transactions['day'] = transactions.date.apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").day)
# transactions['month'] = transactions.date.apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").month)
# transactions['year'] = transactions.date.apply(lambda x: datetime.datetime.strptime(x, "%d.%m.%Y").year)

In [101]:
# Group the data by [day, shop, item and price] to find the cumulative count
trans_a = transactions.groupby(['date', 'date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index(name='item_cnt')

In [104]:
trans_a.head(4)

,date,date_block_num,shop_id,item_id,item_cnt
0,01.01.2013,0,2,991,1.0
1,01.01.2013,0,2,1472,1.0
2,01.01.2013,0,2,1905,1.0
3,01.01.2013,0,2,2920,2.0


In [96]:
# looking at the price variability (in training data) of items which are in the test set.

trans_sub = transactions[transactions.item_id.isin(test.item_id.unique())]
print("total unique items in test set: {}".format(len(test.item_id.unique())))

num_const_price_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 1)
print(num_const_price_items)

price_change1_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 2)
print(price_change1_items)

price_change2_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 3)
print(price_change2_items)

price_change3_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 4)
print(price_change3_items)

price_change4_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 5)
print(price_change4_items)

price_change5_items = sum(trans_sub.groupby('item_id').item_price.nunique() == 6)
print(price_change5_items)

print("\n")

print(num_const_price_items + price_change1_items + price_change2_items +
      price_change3_items + price_change4_items + price_change5_items)

total unique items in test set: 5100
797
584
491
413
361
315


2961


In [168]:

print(shops.shop_name.apply(lambda x: x.split()[0]).unique())
print("\n\n")
print(shops.shop_name.apply(lambda x: x.split()[1]).unique())



SyntaxError: unexpected EOF while parsing (<ipython-input-168-4d935777a393>, line 6)

In [177]:
list(shops.shop_name.apply(lambda x: x.split()[0]).unique())

['!Якутск',
 'Адыгея',
 'Балашиха',
 'Волжский',
 'Вологда',
 'Воронеж',
 'Выездная',
 'Жуковский',
 'Интернет-магазин',
 'Казань',
 'Калуга',
 'Коломна',
 'Красноярск',
 'Курск',
 'Москва',
 'Мытищи',
 'Н.Новгород',
 'Новосибирск',
 'Омск',
 'РостовНаДону',
 'СПб',
 'Самара',
 'Сергиев',
 'Сургут',
 'Томск',
 'Тюмень',
 'Уфа',
 'Химки',
 'Цифровой',
 'Чехов',
 'Якутск',
 'Ярославль']

In [ ]:
### Shops Descriptions:

Выездная Торговля - Outbound Trade

Интернет-магазин ЧС - Online shop Emergency

In [178]:
shops.shop_name.unique() Воронеж

array(['!Якутск Орджоникидзе, 56 фран', '!Якутск ТЦ "Центральный" фран',
       'Адыгея ТЦ "Мега"', 'Балашиха ТРК "Октябрь-Киномир"',
       'Волжский ТЦ "Волга Молл"', 'Вологда ТРЦ "Мармелад"',
       'Воронеж (Плехановская, 13)', 'Воронеж ТРЦ "Максимир"',
       'Воронеж ТРЦ Сити-Парк "Град"', 'Выездная Торговля',
       'Жуковский ул. Чкалова 39м?', 'Жуковский ул. Чкалова 39м²',
       'Интернет-магазин ЧС', 'Казань ТЦ "Бехетле"',
       'Казань ТЦ "ПаркХаус" II', 'Калуга ТРЦ "XXI век"',
       'Коломна ТЦ "Рио"', 'Красноярск ТЦ "Взлетка Плаза"',
       'Красноярск ТЦ "Июнь"', 'Курск ТЦ "Пушкинский"',
       'Москва "Распродажа"', 'Москва МТРЦ "Афи Молл"',
       'Москва Магазин С21', 'Москва ТК "Буденовский" (пав.А2)',
       'Москва ТК "Буденовский" (пав.К7)', 'Москва ТРК "Атриум"',
       'Москва ТЦ "Ареал" (Беляево)', 'Москва ТЦ "МЕГА Белая Дача II"',
       'Москва ТЦ "МЕГА Теплый Стан" II',
       'Москва ТЦ "Новый век" (Новокосино)', 'Москва ТЦ "Перловский"',
       'Москва Т

In [155]:
def getCity(x):
    

ID         0
shop_id    0
item_id    0
dtype: int64